# To run this file, install Python packages: 
- requests
- tabulate

In [2]:
import json
import requests
from random import randrange
from tabulate import tabulate

# Year range
start_year = 2010
end_year = 2020
year = randrange(start_year, end_year)
game_season = str(year) + "-" + str((year + 1) % 100).rjust(2, "0")

# Game type
game_types = ["Regular Season", "Playoffs"]
game_type = game_types[randrange(len(game_types))]

# Game data
players_url = "https://api.pbpstats.com/get-games/nba"
games_params = {
    "Season": game_season,
    "SeasonType": game_type
}
response = requests.get(players_url, params=games_params)
response_json = response.json()["results"]

# Random a game
game_id= randrange(len(response_json))
game_overview = response_json[game_id]
print(json.dumps(game_overview, indent=2))

# Get game stats from the overview
print("Extracting game stats for game:", game_overview["GameId"], "...")
players_url = "https://api.pbpstats.com/get-game-stats"
games_params = {
    "GameId": game_overview["GameId"],
    "Type": "Player" # options are Player, Lineup and LineupOpponent
}
response = requests.get(players_url, params=games_params)
game_stats = response.json()['stats']
print("Done")

{
  "GameId": "0021000794",
  "Date": "2011-02-11",
  "HomeTeamId": "1610612763",
  "AwayTeamId": "1610612749",
  "HomePoints": 89,
  "AwayPoints": 86,
  "HomePossessions": 92,
  "AwayPossessions": 92,
  "HomeTeamAbbreviation": "MEM",
  "AwayTeamAbbreviation": "MIL"
}
Extracting game stats for game: 0021000794 ...
Done


## Some util functions:

In [51]:
# Stats to get
key_general = ["GamesPlayed"]
key_attack_stats = ["Fg2Pct", "Fg3Pct", "FtPoints", "FTA", "Turnovers", "TotalPoss"]
key_defend_stats = ["Steals", "Blocks", "Fouls"]
key_all_stats = key_general + key_attack_stats + key_defend_stats

# Uitil functions
def get_team_players(game_stats, team_type):
    team = []
    if team_type == "Away":
        stats = game_stats["Away"]["FullGame"]
    elif team_type == "Home":
        stats = game_stats["Home"]["FullGame"]

    for entity in stats:
        if entity["Name"] != "Team":
            team.append([entity["Name"], entity["Minutes"]])
    team.sort(key=lambda lst: lst[1], reverse=True)
    return team


def get_player_ids(team, team_type):
    if team_type == "Away":
        team_id = game_overview["AwayTeamId"]
    elif team_type == "Home":
        team_id = game_overview["HomeTeamId"]

    players_url = "https://api.pbpstats.com/get-team-players-for-season"
    players_params = {
        "Season": game_season,
        "SeasonType": game_type,
        "TeamId": team_id
    }
    players_response = requests.get(players_url, params=players_params)
    players = players_response.json()["players"]

    # "team" is list of list, each sub list starts with player name
    for player in team:
        for id, name in players.items():
            if name == player[0]:
                player.append(id)

def get_player_stats_by_id(id):
    url = "https://api.pbpstats.com/get-all-season-stats/nba"
    params = {
        "EntityType": "Player",
        "EntityId": id
    }
    player_stats = requests.get(url, params=params).json()

    for json_object in player_stats["results"][game_type]:
        for key, value in json_object.items():
            if key == "Season" and value == game_season:
                return json_object


def get_team_stats(team):
    for player in team:
        all_stats = get_player_stats_by_id(player[2])
        for key in key_all_stats:
            if key in all_stats:
                player.append(all_stats[key])
            else:
                player.append(None)
        # Add player position
        # player.append(get_player_position(player[2]))

def get_player_position(player_id):
    from nba_api.stats.endpoints import commonplayerinfo
    player_info = commonplayerinfo.CommonPlayerInfo(player_id)
    player_info = player_info.get_normalized_dict()
    # Map the position to 5 short terms: PG, SG, SF, PF, C
    position_map = {
        "Guard": "PG",
        "Point-Guard": "PG",
        "Shooting-Guard": "SG",
        "Forward-Guard": "SG",
        "Forward": "SF",
        "Guard-Forward": "SF",
        "Small-Forward": "SF",
        "Power-Forward": "PF",
        "Center": "C",
        "Forward-Center": "C",
    }
    return position_map[player_info["CommonPlayerInfo"][0]["POSITION"]]

# Print messages
col_names = ["Name", "Minutes", "ID"] + key_all_stats
def print_table(team):
    print(tabulate(team, headers=col_names, tablefmt="grid"))

## Away team stats:

In [4]:
# Away team stats
print("Away team players:")
away_team_players_data = {}
away_team = get_team_players(game_stats, "Away")
get_player_ids(away_team, "Away")
get_team_stats(away_team)
print_table(away_team)

Away team players:
+------------------+-----------+--------+---------------+----------+----------+------------+-------+-------------+-------------+----------+----------+---------+
| Name             | Minutes   |     ID |   GamesPlayed |   Fg2Pct |   Fg3Pct |   FtPoints |   FTA |   Turnovers |   TotalPoss |   Steals |   Blocks |   Fouls |
+==================+===========+========+===============+==========+==========+============+=======+=============+=============+==========+==========+=========+
| Andrew Bogut     | 40:13     | 101106 |            65 | 0.49729  |          |         96 |   217 |         125 |        8502 |       47 |      168 |     217 |
+------------------+-----------+--------+---------------+----------+----------+------------+-------+-------------+-------------+----------+----------+---------+
| Brandon Jennings | 35:38     | 201943 |            63 | 0.422151 | 0.323432 |        199 |   246 |         146 |        8173 |       95 |       21 |     131 |
+--------------

## Home team stats:

In [5]:
# Home team stats
print("Home team players:")
home_team = get_team_players(game_stats, "Home")
get_player_ids(home_team, "Home")
get_team_stats(home_team)
print_table(home_team)

Home team players:
+-----------------+-----------+--------+---------------+----------+----------+------------+-------+-------------+-------------+----------+----------+---------+
| Name            | Minutes   |     ID |   GamesPlayed |   Fg2Pct |   Fg3Pct |   FtPoints |   FTA |   Turnovers |   TotalPoss |   Steals |   Blocks |   Fouls |
+=================+===========+========+===============+==========+==========+============+=======+=============+=============+==========+==========+=========+
| Sam Young       | 42:10     | 201970 |            78 | 0.486486 | 0.34     |         89 |   116 |          61 |        6004 |       68 |       23 |     120 |
+-----------------+-----------+--------+---------------+----------+----------+------------+-------+-------------+-------------+----------+----------+---------+
| Rudy Gay        | 39:43     | 200752 |            54 | 0.485517 | 0.395833 |        194 |   241 |         135 |        8227 |       91 |       58 |     131 |
+-----------------+--

## Aggregate and export the key stats to CSV

In [55]:
import csv

# Get the lineup players key stats
def export_lineup(game_stats, is_home):
    team_type = "Home" if is_home else "Away"
    team_stats = get_team_players(game_stats, team_type)
    get_player_ids(team_stats, team_type)
    get_team_stats(team_stats)
    col_names = ["Name", "Minutes", "ID"] + key_all_stats
    team_stats = [{col_names[i]: player[i] for i in range(len(col_names))} for player in team_stats]
    
    for player in team_stats:
        player["Position"] = get_player_position(player["ID"])
        # Calculate the PassRatio
        # If Turnovers or TotalPoss is None, then use the average value of the team
        if player["Turnovers"] is None or player["TotalPoss"] is None:
            team_turnovers = [player["Turnovers"] for player in team_stats if player["Turnovers"] is not None]
            team_total_poss = [player["TotalPoss"] for player in team_stats if player["TotalPoss"] is not None]
            player["Turnovers"] = sum(team_turnovers) / len(team_turnovers)
            player["TotalPoss"] = sum(team_total_poss) / len(team_total_poss)
        player["PassRatio"] = 1 - player["Turnovers"] / player["TotalPoss"]

    print(tabulate(team_stats[:], headers="keys", tablefmt="grid"))

    positions = ["PG", "SG", "SF", "PF", "C"]
    lineup = {}
    lineup_ids = []
    for position in positions:
        lineup[position] = None
    for player in team_stats:
        if lineup[player["Position"]] is None and player["Position"] in positions:
            lineup[player["Position"]] = player
            lineup_ids.append(player["ID"])

    for position in lineup:
        if lineup[position] is None:
            for player in team_stats:
                if player['ID'] not in lineup_ids:
                    lineup[position] = player
                    lineup_ids.append(player["ID"])
                    break

    print("The lineup of the team:")
    for position in lineup:
        print(position + ":", lineup[position])

    # Export the lineup to a CSV file
    # The columns include position, PassRatio, Fg2Pct, ID, name
    LINEUP_FILE = f"data/lineup_{team_type.lower()}_{game_overview['GameId']}.csv"
    with open(LINEUP_FILE, mode="w") as file:
        writer = csv.writer(file)
        writer.writerow(["Position", "PassRatio", "Fg2Pct", "ID", "Name"])
        for position in lineup:
            player = lineup[position]
            writer.writerow([position, player["PassRatio"], player["Fg2Pct"], player["ID"], player["Name"]])
    
    return lineup

export_lineup(game_stats, is_home=True)
export_lineup(game_stats, is_home=False)



+-----------------+-----------+--------+---------------+----------+----------+------------+-------+-------------+-------------+----------+----------+---------+------------+-------------+
| Name            | Minutes   |     ID |   GamesPlayed |   Fg2Pct |   Fg3Pct |   FtPoints |   FTA |   Turnovers |   TotalPoss |   Steals |   Blocks |   Fouls | Position   |   PassRatio |
+=================+===========+========+===============+==========+==========+============+=======+=============+=============+==========+==========+=========+============+=============+
| Sam Young       | 42:10     | 201970 |            78 | 0.486486 | 0.34     |         89 |   116 |     61      |     6004    |       68 |       23 |     120 | SG         |    0.98984  |
+-----------------+-----------+--------+---------------+----------+----------+------------+-------+-------------+-------------+----------+----------+---------+------------+-------------+
| Rudy Gay        | 39:43     | 200752 |            54 | 0.485517

{'PG': {'Name': 'Mike Conley',
  'Minutes': '38:39',
  'ID': '201144',
  'GamesPlayed': 81,
  'Fg2Pct': 0.46603260869565216,
  'Fg3Pct': 0.3686635944700461,
  'FtPoints': 181,
  'FTA': 247,
  'Turnovers': 176,
  'TotalPoss': 10961,
  'Steals': 144,
  'Blocks': 18,
  'Fouls': 182,
  'Position': 'PG',
  'PassRatio': 0.9839430708876927},
 'SG': {'Name': 'Sam Young',
  'Minutes': '42:10',
  'ID': '201970',
  'GamesPlayed': 78,
  'Fg2Pct': 0.4864864864864865,
  'Fg3Pct': 0.34,
  'FtPoints': 89,
  'FTA': 116,
  'Turnovers': 61,
  'TotalPoss': 6004,
  'Steals': 68,
  'Blocks': 23,
  'Fouls': 120,
  'Position': 'SG',
  'PassRatio': 0.989840106595603},
 'SF': {'Name': 'Darrell Arthur',
  'Minutes': '35:46',
  'ID': '201589',
  'GamesPlayed': 80,
  'Fg2Pct': 0.5016181229773463,
  'Fg3Pct': None,
  'FtPoints': 109,
  'FTA': 134,
  'Turnovers': 88,
  'TotalPoss': 6158,
  'Steals': 52,
  'Blocks': 63,
  'Fouls': 228,
  'Position': 'SF',
  'PassRatio': 0.9857096459889575},
 'PF': {'Name': 'Rudy Gay'